# Irene City
From ancient Greek, *Εἰρήνη*: Peace.

This city finds the signal pulses within the waveforms produced by the
detector or by diomira in the case of Monte Carlo data.

This includes a number of tasks:
- Remove the signal-derivative effect of the PMT waveforms.
- Calibrate PMTs and produced a PMT-summed waveform.
- Remove the baseline from the SiPM waveforms and calibrate them.
- Apply a threshold to the PMT-summed waveform.
- Find pulses in the PMT-summed waveform.
- Match the time window of the PMT pulse with those in the SiPMs.
- Build the PMap object.

## How to use

In your shell, execute
```bash
cd /path/to/IC/work/dir/
source manage.sh work_in_python_version 3.7
```
to activate the *IC-3.7-2020-06-16* conda environment.

Then you are able to run *Irene City* with
```bash
city irene invisible_cities/config/irene.conf
```
being *irene.conf* the configuration for that run.

## Data Transformations

Each of the steps that this algorithm follows are described here. The parameters used in
all python snippets are described in the  [**Keys** section](#keys).

### Raw WaveForm to Corrected WaveForm

The first wave transformation done in *Irene City* is from Raw Waveform
to Corrected Wafevorm. This is made by this snippet
```python
from invisible_cities.dataflow import dataflow as fl
from invisible_cities.cities.components import deconv_pmt

# Raw WaveForm to Corrected WaveForm
rwf_to_cwf = fl.map(deconv_pmt(detector_db, run_number, n_baseline),
                    args = "pmt",
                    out  = "cwf")
```
where the `fl.map` function applies to the `deconv_pmt` function output.


The [`fl.map`](doc/dataflow/map.md) function is described in *doc/dataflow/map.md*
in this project.



## Keys

Let's see what the hack are those configuration keys.

### detector_db

String which always takes
```python
detector_db = "new"
```
Always means, in all configuration files:
```bash
> cat invisible_cities/config/*.conf | grep detector_db
detector_db = 'new'
detector_db = 'new'
detector_db = 'new'
detector_db = 'new'
detector_db = 'new'
detector_db = 'new'
detector_db = 'new'
detector_db = 'new'
detector_db = 'new'
detector_db = 'new'
detector_db = 'new'
detector_db = 'new'
```

Why?

### thr_sipm_type
String to choose how to set the threshold. There are two options for this parameter:
- `"common"`: the threshold is a value in pes.
- `"individual"`: the threshold is a percentual value.

and you can write both in uppercase and lowercase.




## Input parameters
files_in
file_out
compression
event_range
print_mod
detector_db
run_number
n_baseline
n_mau
thr_mau
thr_sipm
thr_sipm_type
s1_lmin
s1_lmax
s1_tmin
s1_tmax
s1_rebin_stride
s1_stride
thr_csum_s1
s2_lmin
s2_lmax
s2_tmin
s2_tmax
s2_rebin_stride
s2_stride
thr_csum_s2
thr_sipm_s2
pmt_samp_wid=25*units.ns
sipm_samp_wid=1*units.mus